# Clasificar dígitos escritos a mano MNIST usando una Red Neuronal Convolucional

### Arquitectura de la Red Neuronal Convolucional

<img src="figuras/ArquitecturaRNC_MNIST.png" width="100%">

Como se puede observar, comenzamos con las imágenes de dígitos en escala de grises MNIST 28 × 28. Luego creamos 32, 5 × 5 filtros convolucionales/canales más activaciones de nodos ReLU. Después de esto, todavía tenemos una altura y un ancho de 28 nodos. A continuación, realizamos una reducción  aplicando una operación de reducción por valor máximo de 2 × 2 con una paso de 2. La capa segunda capa consiste en la misma estructura, pero ahora con 64 filtros/canales y otra reducción por valor máximo con paso de 2. A continuación, aplanamos la salida para obtener una capa completamente conectada con 3164 nodos, seguida de otra capa oculta de 1000 nodos. Estas capas usarán activaciones de nodo ReLU. Finalmente, usamos una capa de clasificación de softmax para dar salida a las probabilidades de los 10 dígitos.

### Entrada de datos y Placeholders

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Hiperparámetors
tasa_de_aprendizaje = 0.0001
epocas = 10
tamaño_lote = 50

# declarar los placeholders de datos para el entrenamiento
# entrada x - de 28 x 28 pixels = 784 - esta es la imagen aplanada obtenida de 
# mnist.train.nextbatch()
x = tf.placeholder(tf.float32, [None, 784])
# reformatear dinamicamente la imagen a dos dimensiones
x_shaped = tf.reshape(x, [-1, 28, 28, 1])
# declarar el placeholder de la salida - 10 dígitos
y = tf.placeholder(tf.float32, [None, 10])

/Users/wladimir/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


TensorFlow tiene un cargador para los datos MNIST. Después de eso, tenemos algunas declaraciones de los hiperparámetros que determinan el comportamiento de optimización (tasa de aprendizaje, tamaño del lote, etc.). A continuación, declaramos un placeholder x para los datos de entrada de la imagen. Los datos de entrada de la imagen se extraerán utilizando la función `mnist.train.nextbatch()`, que proporciona una representación aplanada de la imagen en escala de grises de 28 × 28 = 784 nodos. Sin embargo, antes de poder utilizar estos datos en las funciones de convolución y reducción por valor máximo de TensorFlow, como `conv2d()` y `max_pool()` necesitamos reformar los datos ya que estas funciones solo toman datos 4D.

El formato de los datos a suministrar es `[i, j, k, l]` donde `i` es el número de muestras de entrenamiento, `j` es la altura de la imagen, `k` es el peso y `l` es el número del canal. Debido a que tenemos una imagen en escala de grises, `l` siempre será igual a 1 (si tuviéramos una imagen RGB, sería igual a 3). Las imágenes MNIST son 28 x 28, por lo que `j` y `k` son iguales a 28. Cuando reformamos los datos de entrada `x` en `x_shaped`, teóricamente no conocemos el tamaño de la primera dimensión de x, por lo que no se sabe el valor de `i`. Sin embargo, `tf.reshape()` nos permite poner -1 en lugar de `i` y se cambiará dinámicamente en función del número de muestras de entrenamiento a medida que se realiza el entrenamiento. Entonces se usará `[-1, 28, 28, 1]` para el segundo argumento en `tf.reshape()`.

Finalmente, necesitamos un placeholder para nuestros datos de entrenamiento de salida, que es un tensor de tamaño `[?, 10]`, donde el 10 representa los 10 dígitos posibles para clasificar. Utilizaremos el `mnist.train.next_batch()` para extraer las etiquetas de los dígitos como un vector único; en otras palabras, el dígito "3" se representará como `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`.

### Definir las Capas de Convolución

Debido a que tenemos que crear un par de capas convolucionales, es mejor crear una función para reducir la repetición:

In [2]:
def crear_nueva_capa_conv(datos_entrada, num_canales_entrada, num_filtros, forma_filtro, forma_reducción, nombre):
    # fijar la forma de los filtros de ebtrada para tf.nn.conv_2d
    forma_filtro_conv = [forma_filtro[0], forma_filtro[1], num_canales_entrada,
                      num_filtros]

    # inicializar pesos y sesgos para el filtro
    pesos = tf.Variable(tf.truncated_normal(forma_filtro_conv, stddev=0.03),
                                      name=nombre+'_W')
    sesgo = tf.Variable(tf.truncated_normal([num_filtros]), name=nombre+'_b')

    # fijar la operación de la capa de convolución
    capa_salida = tf.nn.conv2d(datos_entrada, pesos, [1, 1, 1, 1], padding='SAME')

    # agregar el sesgo
    capa_salida += sesgo

    # aplicar la funcion de activación no lineal ReLU
    capa_salida = tf.nn.relu(capa_salida)

    # realizar la reducción por valor máximo
    tamaño = [1, forma_reducción[0], forma_reducción[1], 1]
    pasos = [1, 2, 2, 1]
    capa_salida = tf.nn.max_pool(capa_salida, ksize=tamaño, strides=pasos, 
                               padding='SAME')

    return capa_salida

Crear dos Capas Convolucionales

In [3]:
# crear dos capas convolucionales
capa1 = crear_nueva_capa_conv(x_shaped, 1, 32, [5, 5], [2, 2], nombre='capa1')
capa2 = crear_nueva_capa_conv(capa1, 32, 64, [5, 5], [2, 2], nombre='capa2')

### Capas Completamente Conectadas

Como se discutió previamente, primero tenemos que aplanar la salida de la capa convolucional final. Ahora es una cuadrícula 7 × 7 de nodos con 64 canales, lo que equivale a 3136 nodos por muestra de entrenamiento. Podemos usar `tf.reshape()` para hacer lo que necesitamos:

In [4]:
aplanada = tf.reshape(capa2, [-1, 7 * 7 * 64])

De nuevo, tenemos una primera dimensión calculada dinámicamente (el -1 anterior), que corresponde al número de muestras de entrada en el lote de entrenamiento. A continuación, configuramos la primera capa completamente conectada:

In [5]:
# fijar los pesos y sesgos para esta capa, y después activar con ReLU
wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
capa_densa1 = tf.matmul(aplanada, wd1) + bd1
capa_densa1 = tf.nn.relu(capa_densa1)

Básicamente estamos inicializando los pesos de la capa completamente conectada, multiplicándolos por la salida convolucional aplanada, y luego agregando un sesgo. Finalmente, se aplica una activación de ReLU. La siguiente capa está definida por:

In [6]:
# otra capa con activación softmax
wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev=0.01), name='bd2')
capa_densa2 = tf.matmul(capa_densa1, wd2) + bd2
y_ = tf.nn.softmax(capa_densa2)

Esta capa se conecta a la salida y, por lo tanto, usamos una activación soft-max para producir los valores de salida predichos `y_`. Ahora hemos definido la estructura básica de nuestra red neuronal convolucional. Vamos a definir ahora la función de costo.

### La función de costo de entropía cruzada

Podríamos desarrollar nuestra propia función de costo de entropía cruzada, como hicimos anteriormente, basado en el valor `y_`. Sin embargo, debemos tener cuidado con el manejo de los valores de `NaN`. Afortunadamente, TensorFlow proporciona una función práctica que aplica soft-max seguida de pérdida de entropía cruzada:

In [7]:
entropia_cruzada = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=capa_densa2, labels=y))

La función `softmax_cross_entropy_with_logits()` toma dos argumentos: el primero (`logits`) es el resultado de la multiplicación de la matriz de la capa final (más sesgo) y el segundo es el vector objetivo de entrenamiento. La función primero toma el soft-max de la multiplicación de la matriz, luego lo compara con el objetivo de entrenamiento usando `cross-entropy`. El resultado es el cálculo de entropía cruzada por muestra de entrenamiento, por lo que debemos reducir este tensor en un escalar (un valor único). Para hacer esto usamos `tf.reduce_mean()` que toma la media del tensor.

### El entrenamiento de la red neuronal convolucional

La estructura esencial del entrenamiento de una red neuronal es:
  - Crea un optimizador
  - Crear operaciones predicción correcta y evaluación de precisión
  - Inicializar las operaciones
  - Determine el número de ejecuciones de lotes dentro de una época de entrenamiento
  - Para cada época:
    - Para cada lote:
      - Extraiga los datos del lote
      - Ejecute las operaciones optimizador y entropia_cruzada
      - Agregar al costo promedio
    - Calcule la precisión de la prueba actual
    - Imprime algunos resultados
  - Calcule la precisión de la prueba final e imprima

El código para ejecutar esto es:

In [8]:
# agregar un optimizador
optimizador = tf.train.AdamOptimizer(learning_rate=tasa_de_aprendizaje).minimize(entropia_cruzada)

# definir la operación para evaluar la exactitud
prediccion_correcta = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
exactitud = tf.reduce_mean(tf.cast(prediccion_correcta, tf.float32))

# fijar la operación de inicialización
inicializacion = tf.global_variables_initializer()

with tf.Session() as sess:
    # inicializar las variables
    sess.run(inicializacion)
    total_lotes = int(len(mnist.train.labels) / tamaño_lote)
    for epoca in range(epocas):
        costo_promedio = 0
        for i in range(total_lotes):
            lote_x, lote_y = mnist.train.next_batch(batch_size=tamaño_lote)
            _, c = sess.run([optimizador, entropia_cruzada], 
                            feed_dict={x: lote_x, y: lote_y})
            costo_promedio += c / total_lotes
        exactitud_prueba = sess.run(exactitud, 
                       feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Epoca:", (epoca + 1), "costo =", "{:.3f}".format(costo_promedio), " exactitud prueba: {:.3f}".format(exactitud_prueba))

    print("\nEntrenamiento finalizado!")
    print(sess.run(exactitud, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Epoca: 1 costo = 0.782  exactitud prueba: 0.936
Epoca: 2 costo = 0.154  exactitud prueba: 0.970
Epoca: 3 costo = 0.095  exactitud prueba: 0.978
Epoca: 4 costo = 0.070  exactitud prueba: 0.982
Epoca: 5 costo = 0.057  exactitud prueba: 0.985
Epoca: 6 costo = 0.049  exactitud prueba: 0.986
Epoca: 7 costo = 0.041  exactitud prueba: 0.988
Epoca: 8 costo = 0.035  exactitud prueba: 0.985
Epoca: 9 costo = 0.029  exactitud prueba: 0.989
Epoca: 10 costo = 0.027  exactitud prueba: 0.987

Entrenamiento finalizado!
0.9874
